<a href="https://colab.research.google.com/github/vsnupoudel/DeepLearning-Exercises/blob/main/Keras_Embedding_R_for_Sentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## EDA and starting loading file


In [ ]:
install.packages("FCNN4R", dependencies=TRUE, repos='http://cran.rstudio.com/')

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Warning message:
“package ‘FCNN4R’ is not available for this version of R

A version of this package for your version of R might be available elsewhere,
see the ideas at
https://cran.r-project.org/doc/manuals/r-patched/R-admin.html#Installing-packages”


In [ ]:
library(FCNN4R)

In [ ]:
install.packages( c( "caret","keras","e1071","caTools" )  )
x <- c( "caret","kernlab","e1071","caTools", "keras")
z = lapply(x, require, character.only = TRUE)

Installing packages into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘numDeriv’, ‘SQUAREM’, ‘lava’, ‘prodlim’, ‘iterators’, ‘gower’, ‘ipred’, ‘timeDate’, ‘png’, ‘config’, ‘foreach’, ‘ModelMetrics’, ‘reshape2’, ‘recipes’, ‘pROC’, ‘reticulate’, ‘tensorflow’, ‘tfruns’, ‘zeallot’, ‘proxy’, ‘bitops’


Loading required package: dplyr


Attaching package: ‘dplyr’


The following objects are masked from ‘package:plyr’:

    arrange, count, desc, failwith, id, mutate, rename, summarise,
    summarize


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Loading required package: tidyr

Loading required package: data.table


Attaching package: ‘data.table’


The following objects are masked from ‘package:dplyr’:

    between, first, last


Loading required package: googledrive

Loading required package: caret

Loading required pack

### Downloading the csv files and labels.

In [ ]:
system("gdown --id 1S19nLJ_d0gF7hGyfffOIJvD-F8k4mH7r -O 'tf_idf.csv' ")

In [ ]:
unzip(zipfile= '/content/tf_idf.csv', overwrite = TRUE )

In [ ]:
fill_df1_train_positive <- read.csv('/content/fill_df1_train_positive.csv')

In [ ]:
head(fill_df1_train_positive, 2)

,dim_1,dim_2,dim_3,dim_4,dim_5,dim_6,dim_7,dim_8,dim_9,dim_10,⋯,dim_44,dim_45,dim_46,dim_47,dim_48,dim_49,dim_50,Description,Desc,Target
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<int>
1,-0.51402000,0.637918,0.25157200,0.2834080,0.4950880,0.02275942,-0.0883000,-0.3503385,-0.450752,0.423728,⋯,0.1382978,-0.0498740,0.2458186,0.1186712,-0.4487964,-0.24228080,1.0721700,i love @health4uandpets u guys r the best!!,love healthuandpets u guys r best,4
2,0.08519937,0.407290,0.04108025,-0.1861762,0.3332825,-0.49506000,-0.3306288,0.3591339,-0.347320,0.019373,⋯,0.2087037,0.2583766,-0.1187470,0.1379294,-0.1036104,-0.04515888,0.5299497,im meeting up with one of my besties tonight! cant wait!! - girl talk!!,im meeting one besties tonight cant wait girl talk,4


In [ ]:
fill_df1_train_negative <- read.csv('/content/fill_df_train_negative.csv')
head(fill_df1_train_negative,2)

,dim_1,dim_2,dim_3,dim_4,dim_5,dim_6,dim_7,dim_8,dim_9,dim_10,⋯,dim_44,dim_45,dim_46,dim_47,dim_48,dim_49,dim_50,Description,Desc,Target
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<int>
1,-0.11758273,0.05472664,0.1435187,-0.04461991,-0.07263091,-0.1293335,-0.2996061,0.14176000,-0.1662370,0.170450,⋯,-0.0006929091,0.002369427,0.03921009,0.1057899,-0.2360387,-0.1430836,0.7741466,"@switchfoot http://twitpic.com/2y1zl - awww, that's a bummer. you shoulda got david carr of third day to do it. ;d",switchfoot httptwitpiccomyzl awww thats bummer shoulda got david carr third day d,0
2,-0.08274296,0.11447275,0.2776635,-0.04677667,-0.08026000,-0.1987698,-0.2196217,-0.07718792,-0.2282217,0.207352,⋯,0.3245757250,0.338379179,0.09025933,0.0563986,-0.0191855,0.1115213,0.1747626,is upset that he can't update his facebook by texting it... and might cry as a result school today also. blah!,upset cant update facebook texting might cry result school today also blah,0


In [ ]:
#Read df into one
raw_data <- rbind(fill_df1_train_positive, fill_df1_train_negative)
# Remove any row that has NA
nonNA_data <-  na.omit(raw_data) 
# change target value 4 to 1
nonNA_data['Target'][nonNA_data['Target'] == 4] <- 1
# Start trying models
model_data <- nonNA_data[, c(c(1:50), 53) ]
model_data['Target'] <- as.factor( as.integer( model_data$Target ) )

In [ ]:
# Training our first model 
library(caret)
sample = createDataPartition(
  model_data$Target  ,
  p = 0.9, list=FALSE
)
training <- model_data[sample,]
testing <- model_data[-sample, ]

In [ ]:
## Start training
fitControl <- trainControl(## 10-fold CV
  method = "repeatedcv",
  number = 5,
  ## repeated ten times
  repeats = 5)

start_time <- Sys.time()
modSVMrad <- train( Target~., data=training
                    , method="mlpSGD" 
                    , preProcess = c("center", "scale", "pca")
                    ,  trControl = fitControl)
end_time <- Sys.time()
end_time - start_time

1 package is needed for this model and is not installed. (FCNN4R). Would you like to try to install it now?

ERROR: ignored

In [ ]:
## Predict and calculate test set accuracy
predRF <- predict(modSVMrad, newdata= testing )
tableRF <- table(predRF, testing$Target )
tableRF

      
predRF      0      1
     0 106693  49081
     1  52062 121746

In [ ]:
# Accuracy
sum(diag(tableRF))/ sum(tableRF) 

[1] 0.6931173